In [7]:
import time
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

mp.cpu_count()

4

In [8]:
from fencoding_CPUs import (save_to_csv,
                               reduce_mem_usage,
                               EmptyElim,
                               FEncoding,
                               OutlDetect,
                               FImputation
                               )

In [9]:
from pathlib import Path
# home-credit-default-risk tables
if not Path("data/application_train.csv").is_file():
    ! wget https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0 -O data.zip
    ! unzip -q data.zip
    ! rm data.zip
# default-of-credit-card-clients-dataset
if not Path("data/default_ucr.csv").is_file():
    ! wget https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0 -O default_ucr.csv

In [10]:
# Read in the source datasets
dict = {
    'datasets':[
                # home-credit-default-risk tables
                pd.read_csv('data/application_train.csv'),
                # default-of-credit-card-clients-datasets
                pd.read_csv('data/default_ucr.csv'),
    ],
    'name_dropped_columns':
                [
                 # home-credit-default-risk tables
                 ['SK_ID_CURR', 'TARGET'],
                 # default-of-credit-card-clients-datasets
                 ['ID', 'default.payment.next.month']              
    ],   
}
# Keep ID and target columns separately
dict['dropped_columns'] = [dict['datasets'][i][dict['name_dropped_columns'][i]] for i in range(len(dict['datasets']))]
# Drop ID and target columns from the tables
dict['datasets'] = [dict['datasets'][i].drop(dict['name_dropped_columns'][i], axis=1) for i in range(len(dict['datasets']))]

In [12]:
# default-of-credit-card-clients-datasets
X, y = dict['datasets'][1], dict['dropped_columns'][1]['default.payment.next.month']
X = reduce_mem_usage(X)
#fencoding.initialize_types(X)


Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.77 MB
Memory usage decreased by 66.3%


In [11]:
emptyelim = EmptyElim(n_jobs= -1, chunks = None)
fencoding = FEncoding(n_jobs = -1, chunks = None)
outldetect = OutlDetect(outliers_detection_technique = 'iqr_proximity_rule', n_jobs = -1, 
                                                                             chunks = None)

# For regression
fimputation_rb = FImputation('regression-based', fill_with_value = None, n_jobs = -1, chunks = None)
# For boosting
fimputation_tb = FImputation('tree-based', fill_with_value = 'extreme_values', n_jobs = -1, chunks = None)

In [13]:
X = emptyelim.fit_transform(X)
X = fencoding.date_replace(X)
X = fencoding.encode_categor(X, method = 'OrdinalEncoder')
X = outldetect.fit_transform(X)


 columns to drop: {}

 No time columns in the dataset

 No numerical columns in the dataset

 No numerical columns in the dataset


In [14]:
XX = fimputation_rb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/default_ucr_regression.csv')

XX = fimputation_tb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/default_ucr_boosting.csv')


Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 81.0%

 Successfully saved to data/default_ucr_regression.csv

Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 81.0%

 Successfully saved to data/default_ucr_boosting.csv


In [15]:
# home-credit-default-risk tables
X, y = dict['datasets'][0], dict['dropped_columns'][0]['TARGET']
X = reduce_mem_usage(X)
#fencoding.initialize_types(X)


Memory usage of dataframe is 90.91 MB
Memory usage after optimization is: 90.91 MB
Memory usage decreased by 0.0%


In [15]:
emptyelim = EmptyElim(n_jobs= -1, chunks = None)
fencoding = FEncoding(n_jobs = -1, chunks = None)
outldetect = OutlDetect(outliers_detection_technique = 'iqr_proximity_rule', n_jobs = -1, 
                                                                             chunks = None)

# For regression
fimputation_rb = FImputation('regression-based', fill_with_value = None, n_jobs = -1, chunks = None)
# For boosting
fimputation_tb = FImputation('tree-based', fill_with_value = 'extreme_values', n_jobs = -1, chunks = None)

In [16]:
X = emptyelim.fit_transform(X)
X = fencoding.date_replace(X)
X = fencoding.encode_categor(X, method = 'OrdinalEncoder')
X = outldetect.fit_transform(X)


 columns to drop: {}

 No time columns in the dataset

 No numerical columns in the dataset

 No numerical columns in the dataset


In [17]:
XX = fimputation_rb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/application_train_regression.csv')

XX = fimputation_tb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/application_train_boosting.csv')


Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 81.0%

 Successfully saved to data/application_train_regression.csv

Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 81.0%

 Successfully saved to data/application_train_boosting.csv
